### Combining Model Result

In [1]:
import numpy as np
import pandas as pd

In [20]:
def model_blending(model_list,model_weight):
    
    result = model_list[0]*model_weight[0]
    N = len(model_list)
    
    for i in range(1,N):
        result += model_list[i]*model_weight[i]
    
    result = result/np.sum(model_weight)
    
    return result

In [8]:
def read_submission(submission_list):
    data_path = './zillow data/draft_submission'
    model_list = []
    for i in submission_list:
        model = pd.read_csv(data_path +('%d.csv'%i),index_col=0)
        model_list.append(model.copy())
    
    return model_list

In [9]:
model_list = read_submission([5,8,9])

/Users/apple/venv/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [25]:
weight = np.ones(len(model_list))
result = model_blending(model_list, weight)

In [27]:
result.to_csv("./zillow data/combine_submission1.csv")

### Blending Methods

In [16]:
import numpy as np
from sklearn.cross_validation import KFold
import pandas as pd

In [ ]:
np.random.seed(0)  # seed to shuffle the train set
n_folds = 10
verbose = True
shuffle = False

if shuffle:
    idx = np.random.permutation(y.size)
    X = X[idx]
    y = y[idx]
    
skf = list(KFold(y, n_folds))

models = []

print "Creating train and test sets for blending."
dataset_blend_train = np.zeros((X.shape[0], len(models)))
dataset_blend_test = np.zeros((X_submission.shape[0], len(models)))

for j, model in enumerate(models):
    print(j, model)
    dataset_blend_test_j = np.zeros((X_submission.shape[0], len(skf)))
        for i, (train, test) in enumerate(skf):
            print("Fold", i)
            X_train = X[train]
            y_train = y[train]
            X_test = X[test]
            y_test = y[test]
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            dataset_blend_train[test, j] = y_pred
            dataset_blend_test_j[:, i] = model.predict(X_submission)
        dataset_blend_test[:, j] = dataset_blend_test_j.mean(1)
    

print("Blending.")
model = LogisticRegression()
model.fit(dataset_blend_train, y)
y_submission = model.predict(dataset_blend_test)

print("Saving Results.")


### Stacking Methods

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from vecstack import stacking

In [ ]:
models = [
    ExtraTreesRegressor(random_state=0, n_jobs=-1, 
                        n_estimators=100, max_depth=3),
        
    RandomForestRegressor(random_state=0, n_jobs=-1, 
                          n_estimators=100, max_depth=3),
        
    XGBRegressor(random_state=0, n_jobs=-1, learning_rate=0.1, 
                 n_estimators=100, max_depth=3)
]

In [ ]:
S_train, S_test = stacking(models,                     # list of models
                           X_train, y_train, X_test,   # data
                           regression=True,            # regression task (if you need 
                                                       #     classification - set to False)
                           mode='oof_pred_bag',        # mode: oof for train set, predict test 
                                                       #     set in each fold and find mean
                           save_dir=None,              # do not save result and log (to save 
                                                       #     in current dir - set to '.')
                           metric=mean_absolute_error, # metric: callable
                           n_folds=4,                  # number of folds
                           shuffle=True,               # shuffle the data
                           random_state=0,             # ensure reproducibility
                           verbose=2)  

In [ ]:
# Initialize 2nd level model
model = XGBRegressor(random_state=0, n_jobs=-1, learning_rate=0.1, 
                     n_estimators=100, max_depth=3)
    
# Fit 2nd level model
model = model.fit(S_train, y_train)

# Predict
y_pred = model.predict(S_test)

# Final prediction score
print('Final prediction score: [%.8f]' % mean_absolute_error(y_test, y_pred))
